# **Spam Ham Detection Using BERT and Tensorflow**

##### <u>**Project Summary**</u>

##### <u>**GitHub Link**</u>
[Click Here](https://github.com/ajitmane36/spam-ham-detection-bert-tensorflow.git)

##### <u>**Problem Statement**</u>

- The data is related to the classification of emails into spam or ham (non-spam). The goal of this project is to develop a model using BERT and TensorFlow to predict whether an email is spam or not based on its content. By fine-tuning a pre-trained BERT model, the objective is to enhance the accuracy and efficiency of email classification, ensuring that legitimate emails are delivered to the inbox while spam is effectively filtered out.

##### <u>**Data Description**</u>

- **Message**: Description of the email content (text).
- **Category**: Indicates whether the email is spam (1) or not (0).

In [7]:
!pip install tensorflow_text

ERROR: Could not find a version that satisfies the requirement tensorflow_text (from versions: none)
ERROR: No matching distribution found for tensorflow_text


In [37]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Importing tensorflow libraries
import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_text as text

# filter warnings
import warnings
warnings.filterwarnings('ignore')

ModuleNotFoundError: No module named 'tensorflow_text'

In [ ]:
# Dataset Loading
df=pd.read_csv(r"C:\Users\ajitm\Downloads\DS Projects\Deep Larning Projects\1. Text Classification Using BERT & Tensorflow\spam_emails_data.csv")
df

In [ ]:
# Fist five observations
df.head()

In [ ]:
# Last five observations
df.tail()

##### <u>**Data Inispection**</u>

In [ ]:
# Shape of dataset
df.shape
print(f'Dataset has {df.shape[0]} observations and {df.shape[1]} columns.')

In [ ]:
# Dataset columns
print(df.columns.tolist())

In [ ]:
# Basic information of dataset
df.info()

In [ ]:
# Basic description of dataset
df.groupby('Category').describe()

- Dataset having 4825 Ham observations and 747 spam observations.
- Class imbalance seen.

##### <u>**Data Wrangling**</u>

In [ ]:
# Cehcking  duplicates value in each feature
duplicates_df=pd.DataFrame({'columns':df.columns, 'number_of_duplicates': df.duplicated().sum()}).sort_values(by='number_of_duplicates', ascending=False)
print(duplicates_df)
print(f'Dataset having {df.duplicated().sum()} duplicates values.')

In [ ]:
# Removing duplicates
df.drop_duplicates(inplace=True)
df.shape

In [ ]:
# Checking missing values
null_df=pd.DataFrame({'columns': df.columns, 'num_of_nulls': df.isna().sum()})
print(null_df )
print(f'Dataset have {df.isna().sum()} null values.')

##### <u>**Exploratory Data Analysis**</u>

In [ ]:
# Min and Max length of email
print("Smallest email")
print('__'*50)
print(df.Message.min())
print(f'Length: {len(df.Message.min())}')
print('=='*50)
print("Largest email")
print('__'*50)
print(df.Message.max())
print(f'Length: {len(df.Message.max())}')

In [ ]:
# Defining a function to Notate the percent count of each value on the bars
def annot_percent(axes):
    '''Takes axes as input and labels the percent count of each bar in a countplot'''
    for p in plot.patches:
        total = sum(p.get_height() for p in plot.patches)/100
        percent = round((p.get_height()/total),2)
        x = p.get_x() + p.get_width()/2
        y = p.get_height()
        plot.annotate(f'{percent}%', (x, y), ha='center', va='bottom')

In [ ]:
# Bar plot for diffrent categories of email 
plt.figure(figsize=(7,5))
plot=plt.subplot(111)
ax=sns.countplot(x=df.Category, hue=df.Category)
ax.set_title('Countplot for different emails categories')
ax.legend('Category', title='Emails Categories')
annot_percent(plot)
plt.tight_layout()
plt.show()

- Dataset have 12.43% of spam and 87.57% ham observations.
- Class imbalnce seen.

In [ ]:
# Length of each emial
df['length']=df['Message'].apply(lambda x:len(x))
df.head()

In [ ]:
# Plot histogram for length of emails
plt.figure(figsize=(7,5))
ax=sns.histplot(x=df.length, hue=df.Category)
ax.set_title('Histogram for length of emails')
ax.axvline(df.length.mean(), color='r', linestyle='--')
ax.text(df.length.mean(), ax.get_ylim()[1]*0.9, f'Average Emails Length: {df.length.mean():.2f}', 
        color='red', ha='left', va='top', fontsize=12)
plt.tight_layout()
plt.show()

- The average email length seen is 79.
- The majority of emails are shorter than 200 characters.

##### <u>**Feature Engineering & Data Pre-processing**</u>

In [ ]:
# Dataset look
df

In [ ]:
# Dropping unnecessary 'length' feature
df.drop(columns='length', inplace=True)
df

In [ ]:
# Check category in 'Category' feature
df.Category.unique()

In [ ]:
# Categorical encoding for 'Category' feature
df['spam']=df['Category'].apply(lambda x: 1 if x=='spam' else 0)
df

In [ ]:
# Separating dependant and independant varibale

# Independant feature
X=df['Message'].copy()
# Dependant feature
y=df['spam'].copy()

In [ ]:
# Splitting dataset into train and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=42, stratify=y)

In [ ]:
 # Shape of splits
print(f'X_train: {X_train.shape}')
print(f'X_test: {X_test.shape}')

In [43]:
# Download BERT preprocessing and encoder model from tensorflow hub
preprocessor = hub.KerasLayer(
    "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3")
encoder = hub.KerasLayer(
    "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-12-h-768-a-12/2",
    trainable=True)

RuntimeError: Op type not registered 'CaseFoldUTF8' in binary running on AJITS-PC. Make sure the Op and Kernel are registered in the binary running in this process. Note that if you are loading a saved graph which used ops from tf.contrib (e.g. `tf.contrib.resampler`), accessing should be done before importing the graph, as contrib ops are lazily registered when the module is first accessed.